In [1]:
from numpy import loadtxt
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Add
import keras
from keras.utils import plot_model
import keras.backend as K
import tensorflow as tf

Using TensorFlow backend.
/home/icomp/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/icomp/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/icomp/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/icomp/anaconda3/lib/python3.7/site-packages/tensorflo

In [2]:
#ajustando a seed para gerar pesos aleatórios
np.random.seed(1)
tf.random.set_random_seed(1)

In [3]:
#carregendo os dados
dataset = np.loadtxt('trains.csv', delimiter=',')

#separando os dados dos labels
X = dataset[:,:-1]
y = dataset[:,-1]

In [74]:
print(X)
print(y)

[[ 5.  4.  2.  1.  9.  3.  4.  2. -1.  5.  1.  1.  3.  1.  9.  1.  3.  2.
  -1.  9.  1.  2. -1.  1. -1. -1. -1.  1. -1. -1.  1. -1.]
 [ 4.  3.  2. -1.  1.  1.  1.  2. -1.  2.  1.  4.  2. -1.  4.  2.  2.  0.
   0.  0.  0.  0. -1.  1. -1.  1. -1. -1. -1. -1. -1. -1.]
 [ 4.  2.  2. -1.  9.  1.  2.  2. -1.  6.  1.  1.  3.  1.  4.  1.  1.  0.
   0.  0.  0.  0. -1. -1. -1. -1.  1. -1.  1. -1. -1. -1.]
 [ 5.  2.  2. -1.  2.  1.  1.  2. -1.  3.  1.  1.  2. -1.  8.  1.  4.  2.
  -1.  9.  1.  4.  1.  1. -1. -1.  1. -1. -1. -1. -1. -1.]
 [ 4.  3.  2. -1.  3.  1.  1.  3.  1.  4.  1.  4.  2. -1.  4.  1.  2.  0.
   0.  0.  0.  0. -1.  1. -1.  1. -1. -1. -1. -1. -1. -1.]
 [ 3.  2.  2.  1.  4.  3.  2.  2. -1.  9.  1.  1.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0. -1. -1. -1. -1. -1. -1.  1. -1. -1. -1.]
 [ 4.  2.  2. -1.  3.  1.  2.  2. -1.  1.  1.  1.  2.  1.  7.  0.  0.  0.
   0.  0.  0.  0. -1. -1. -1. -1. -1. -1.  1. -1. -1. -1.]
 [ 3.  2.  3.  1.  4.  1.  4.  2. -1.  1.  1.  2.  0.  0.  0.  0.  0.

In [5]:
#construindo nosso modelo para a primeira rede

ip = Input(shape=(32,)) #neurônios de entrada
x = Dense(9, activation='tanh', kernel_initializer='he_normal')(ip) #Camada escondida
out =  Dense(1, activation='tanh')(x) #camada de saída
model = Model(outputs=[out],inputs=[ip])

In [6]:
opt = keras.optimizers.SGD(learning_rate=0.3, momentum=0.4, nesterov=False)
model.compile(loss='mse', optimizer=opt, metrics=['mse'])
model.fit(X,y, epochs=10000, batch_size=9,validation_split=0.1, verbose=0)
model.predict(X) #predizendo os exemplos usados no treino...

array([[ 0.9999757 ],
       [ 0.99464077],
       [ 0.9947883 ],
       [ 0.9992981 ],
       [ 0.9948395 ],
       [-0.99617064],
       [-0.9999245 ],
       [-0.99602324],
       [-0.99192154],
       [-0.9999997 ]], dtype=float32)

In [7]:
# preparação ppara fazer leave one out cross-validation
lx = []
ly = []
X2 = [list(x) for x in X]
y2 = [float(x) for x in y]



for i in range(len(X2)):
    lx.append(X2[:i]+[X2[k] for k in range(len(X2)) if i < k]+[X2[i]])
    ly.append(y2[:i]+[y2[k] for k in range(len(y2)) if i < k]+[y2[i]])

## Primeira rede

Feita com KBANN (pesos aleatórios) e validada com leave one out crossvalidation.

Funções de ativação: tanh

Accuracy = 40%

In [8]:
predictions = []
for i in range(len(lx)):
    ip = Input(shape=(32,)) #neurônios de entrada
    x = Dense(9, activation='tanh', kernel_initializer='he_normal')(ip) #Camada escondida
    out =  Dense(1, activation='tanh')(x) #camada de saída
    model = Model(outputs=[out],inputs=[ip])
    opt = keras.optimizers.SGD(learning_rate=0.3, momentum=0.4, nesterov=False)
    model.compile(loss='mse', optimizer=opt, metrics=['mse'])
    model.fit(np.array(lx[i])[:-1:], np.array(ly[i])[:-1:], epochs=10000, batch_size=9, verbose=0)
    predictions.append(model.predict(np.array(lx[i])[-1::])[0][0])
    

In [9]:
for x in range(len(predictions)):
    print(predictions[x], "(expected:",y[x],")")

0.9295803 (expected: 1.0 )
0.598682 (expected: 1.0 )
-0.99264264 (expected: 1.0 )
0.964253 (expected: 1.0 )
0.0065902695 (expected: 1.0 )
0.7987824 (expected: -1.0 )
0.66667014 (expected: -1.0 )
0.99148434 (expected: -1.0 )
-0.32639512 (expected: -1.0 )
-0.9991104 (expected: -1.0 )


In [ ]:
#plot

## Segunda rede

Feita com proposicionalização e validada com leave one out crossvalidation.

Os pesos representam a influência de um determinado neurônio de entrada na preposição que gerou aquele neurônio.

O peso 3.5 e os bias foram calculados segundo [1].

Funções de ativação: sigmoid para a camada escondida e tanh para a saída.

Accuracy: 60%




[1] Garcez, A. and Zaverucha, G. (1999). The connectionist inductive learning and
    logic programming system. Applied Intelligence Journal, 11(1), pp. 59-77. doi:
    10.1023/A:1008328630915

In [65]:
weights = [[1,0,1,1,1,1,1,1,-1,1,1,1,1,1,1,1,1,1,-1,1,1,1,0,0,0,0,0,0,0,0,0,0],
           [1,0,1,-1,1,1,1,1,-1,1,1,1,1,-1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],\
           [1,0,1,-1,1,1,1,1,-1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],\
           [1,0,1,-1,1,1,1,1,-1,1,1,1,1,-1,1,1,1,1,-1,1,1,1,0,0,0,0,0,0,0,0,0,0],\
           [1,0,1,-1,1,1,1,1,1,1,1,1,1,-1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],\
           [1,0,1,1,1,1,1,1,-1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],\
           [1,0,1,-1,1,1,1,1,-1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],\
           [1,0,1,1,1,1,1,1,-1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],\
           [1,0,1,-1,1,1,1,1,1,1,1,1,1,-1,1,1,1,1,-1,1,1,1,0,0,0,0,0,0,0,0,0,0],\
           [1,0,1,-1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]

weights_out = [[1],[1],[1],[1],[1],[-1],[-1],[-1],[-1],[-1]]

weights = [[k*(3.5) for k in i] for i in weights]
weights_out = [[k*(3.5) for k in i] for i in weights_out]


bias = [[1.32],\
       [0.99],\
       [0.99],\
       [1.32],\
       [0.99],\
       [0.66],\
       [0.99],\
       [0.66],\
       [1.32],\
       [0.66]]

bias_out = [[-0.96]]


print(weights_out)

[[3.5], [3.5], [3.5], [3.5], [3.5], [-3.5], [-3.5], [-3.5], [-3.5], [-3.5]]


In [66]:
lw = []
lw_out = []
lb = []
for i in range(len(weights)):
    lw.append(weights[:i:]+[weights[k] for k in range(len(weights)) if k > i]+[weights[i]])
    lw_out.append(weights_out[:i:]+[weights_out[k] for k in range(len(weights_out)) if k > i]+[weights_out[i]])
    lb.append(bias[:i:]+[bias[k] for k in range(len(bias)) if k > i]+[bias[i]])
    

In [67]:
def lw_init(shape, dtype=None):
    global i,lw
    #print(np.array(lw[i][:-1:], dtype=dtype).T)
    return np.array(lw[i][:-1:], dtype=dtype).T

def lb_init(shape, dtype=None):
    global i,lb
    #print(np.array(lb[i][:-1:], dtype=dtype).T[0])
    return np.array(lb[i][:-1:], dtype=dtype).T[0]

def lw_out_init(shape, dtype=None):
    global i,lw_out
    #print(np.array(lw_out[i][:-1:], dtype=dtype))
    return np.array(lw_out[i][:-1:], dtype=dtype)

def bias_out_init(shape, dtype=None):
    global i,lb
    #print(np.array(bias_out[0], dtype=dtype))
    return np.array(bias_out[0], dtype=dtype)

In [107]:
predictions = []
global i
for i in range(len(lx)):
    ip = Input(shape=(32,)) #neurônios de entrada
    x = Dense(9, activation='sigmoid',\
             kernel_initializer=lw_init,\
             bias_initializer=lb_init)(ip) #Camada escondida
    
    out =  Dense(1, activation='tanh',\
                 kernel_initializer=lw_out_init,\
                 bias_initializer=bias_out_init)(x) #camada de saída
    
    model = Model(outputs=[out],inputs=[ip])
    opt = keras.optimizers.SGD(learning_rate=0.3, momentum=0.4, nesterov=False)
    model.compile(loss='mse', optimizer=opt, metrics=['mse'])
    model.fit(np.array(lx[i])[:-1:], np.array(ly[i])[:-1:], epochs=10000, batch_size=1,verbose=0)
    predictions.append(model.predict(np.array(lx[i])[-1::])[0][0])

In [108]:
for x in range(len(predictions)):
    print(predictions[x], "(expected:",y[x],")")

-1.0 (expected: 1.0 )
1.0 (expected: 1.0 )
-1.0 (expected: 1.0 )
-1.0 (expected: 1.0 )
1.0 (expected: 1.0 )
1.0 (expected: -1.0 )
1.0 (expected: -1.0 )
1.0 (expected: -1.0 )
-1.0 (expected: -1.0 )
1.0 (expected: -1.0 )


In [105]:
model.layers[2].get_weights()



[array([[ 2.3471217],
        [ 2.3471217],
        [ 2.3471217],
        [ 2.3471217],
        [ 2.3471217],
        [-4.6528544],
        [-4.6528544],
        [-4.6528544],
        [-4.6528544],
        [-4.6528544]], dtype=float32), array([-2.112987], dtype=float32)]

In [113]:
pip3 install pydot --user

SyntaxError: invalid syntax (<ipython-input-113-e523bdf51e29>, line 1)

In [116]:
#plot
plot_model(model, show_shapes=True, show_layer_names=False)

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.

## Extraindo regras da segunda rede

Primeiramente treinamos ela novamente, agora com todos os 10 exemplos. Em seguida observamos os pesos encontrados após o processo de aprendizagem para descobrir qual a influência das cláusulas no resultado.

In [96]:
def lw_init(shape, dtype=None):
    global weights
    return np.array(weights, dtype=dtype).T

def lb_init(shape, dtype=None):
    global bias
    return np.array(bias, dtype=dtype).T[0]

def lw_out_init(shape, dtype=None):
    global weights_out
    return np.array(weights_out, dtype=dtype)

def bias_out_init(shape, dtype=None):
    global bias_out
    return np.array(bias_out[0], dtype=dtype)

ip = Input(shape=(32,)) #neurônios de entrada
x = Dense(9, activation='tanh',\
             kernel_initializer=lw_init,\
             bias_initializer=lb_init)(ip) #Camada escondida
    
out =  Dense(1, activation='tanh',\
                 kernel_initializer=lw_out_init,\
                 bias_initializer=bias_out_init)(x) #camada de saída
    
model = Model(outputs=[out],inputs=[ip])
opt = keras.optimizers.SGD(learning_rate=0.3, momentum=0.4, nesterov=False)
model.compile(loss='mse', optimizer=opt, metrics=['mse'])
model.fit(X, y, epochs=10000, batch_size=1,verbose=0)
predictions= [i for i in model.predict(X)]

Exception ignored in: <function BaseSession._Callable.__del__ at 0x7ff0b7a39b00>
Traceback (most recent call last):
  File "/home/icomp/anaconda3/lib/python3.7/site-packages/tensorflow/python/client/session.py", line 1473, in __del__
    self._session._session, self._handle)
KeyboardInterrupt


KeyboardInterrupt: 

In [79]:
for x in range(len(predictions)):
    print(predictions[x], "(expected:",y[x],")")

[1.] (expected: 1.0 )
[1.] (expected: 1.0 )
[1.] (expected: 1.0 )
[1.] (expected: 1.0 )
[1.] (expected: 1.0 )
[1.] (expected: -1.0 )
[1.] (expected: -1.0 )
[1.] (expected: -1.0 )
[1.] (expected: -1.0 )
[1.] (expected: -1.0 )
